In [1]:
# Setup dependencies (as taken from assignment 6)
import os
import math
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models

from torchsummary import summary
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

#Additional Setup to use Tensorboard
!pip install -q tensorflow
%load_ext tensorboard

In [2]:
df = pd.read_csv('day_ahead.csv')
df = df.set_index('datetime')
df.index = pd.to_datetime(df.index)
df

,Day-ahead Price [EUR/MWh],tempC,windspeedKmph,winddirDegree,precipMM,humidity,pressure,time_increment
datetime,,,,,,,,
2015-01-05 00:00:00,-0.243620,-1.722376,-0.422598,0.563889,0.000000,0.96,2.163731,2.4
2015-01-05 01:00:00,-0.316395,-1.722376,-0.297098,0.572222,0.000000,0.96,2.058525,0.1
2015-01-05 02:00:00,-0.439933,-1.722376,-0.171598,0.580556,0.000000,0.96,2.058525,0.2
2015-01-05 03:00:00,-0.625914,-1.722376,-0.046098,0.588889,0.000000,0.96,1.953318,0.3
2015-01-05 04:00:00,-0.626363,-1.722376,-0.171598,0.575000,0.000000,0.96,1.953318,0.4
...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,0.877205,-1.130468,-0.673599,0.602778,0.054545,0.94,-1.728907,1.9
2020-12-31 20:00:00,0.665169,-1.130468,-0.422598,0.625000,0.036364,0.93,-1.728907,2.0
2020-12-31 21:00:00,0.469755,-1.130468,-0.171598,0.650000,0.054545,0.93,-1.623701,2.1


# Create Torch dataset

In [3]:
# sequence length (edit the value for different sequence length)
seq = 36 

In [4]:
delta = pd.Timedelta(seq, unit ='h')
# define 1 hour object for convenience when using datetime as index in the dataframe to not include the last item
hours_12 = pd.Timedelta(12, unit ='h') # used mostly for empty 12 hours 
hour = pd.Timedelta(1, unit ='h')
day = pd.Timedelta(1, unit ='d')

In [5]:
### creating training dataset
train_y_start = dt.datetime(2015, 1, 5, 0, 0) + (delta+hours_12).ceil('1d')
#train_x_start = train_y_start - delta - hours_12
train_end = dt.datetime(2020, 11, 30, 23, 0)

train_x = []
train_y = []
while train_y_start + day - hour <= train_end:
    train_x_start = train_y_start - delta - hours_12
    
    
    #print(train_x_start, train_y_start)
    train_x.append(df[train_x_start:train_x_start+delta - hour].values)
    train_y.append(df[train_y_start:train_y_start+day - hour]['Day-ahead Price [EUR/MWh]'].values)
    
    train_y_start += day
    
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
print(train_x.shape)
print(train_y.shape)

(2155, 36, 8)
(2155, 24)


In [6]:
### creating testing dataset
test_y_start = dt.datetime(2020, 12, 1, 0, 0)
test_end = dt.datetime(2020, 12, 31, 23, 0)

test_x = []
test_y = []
while test_y_start + day - hour <= test_end:
    test_x_start = test_y_start - delta - hours_12
    
    test_x.append(df[test_x_start:test_x_start+delta - hour].values)
    test_y.append(df[test_y_start:test_y_start+day - hour]['Day-ahead Price [EUR/MWh]'].values)
    
    test_y_start += day

test_x = np.asarray(test_x)
test_y = np.asarray(test_y)
print(test_x.shape)
print(test_y.shape)

(31, 36, 8)
(31, 24)


In [7]:
# create tensor objects
x_train = torch.from_numpy(train_x).float()
y_train = torch.from_numpy(train_y).float()
x_test = torch.from_numpy(test_x).float()
y_test = torch.from_numpy(test_y).float()

# Define BLSTM model

In [8]:
class BLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(BLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, output_dim) # multiply hidden_dim by 2 because bidirectional
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).requires_grad_() #hidden layer output
        # Initialize cell state
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).requires_grad_() 
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        # Index hidden state of last time step
        out = self.fc(out[:, -1, :]) 
        return out

## Training

In [57]:
num_train = x_train.shape[0]
input_dim = x_train.shape[2]
output_dim = 24 
hidden_dim = 20 # no. of neurons in hidden layer
num_layers = 3 # no of hidden layers 
num_epochs = 10

In [58]:
model = BLSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
# for practice use MSE, in real experiment use NLLLOSS for parametric
criterion = nn.MSELoss(reduction='mean')
#criterion = nn.NLLLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [59]:
# training loop
for t in range(num_epochs): 
    err = 0
    for i in range(num_train):
        y_train_pred = model(x_train[i].unsqueeze(0))
        loss = criterion(y_train_pred, y_train[i])
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        err += loss.item()
        #print("item ", t, "MSE: ", loss.item())
        
    print("Epoch ", t, "MSE: ", err/num_train)

Epoch  0 MSE:  0.6251276288942159
Epoch  1 MSE:  0.6146098782790267
Epoch  2 MSE:  0.6003325983874388
Epoch  3 MSE:  0.5674329914341696
Epoch  4 MSE:  0.5604366081684672


KeyboardInterrupt: 

In [ ]:
# training old loop
for t in range(num_epochs): 
    
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train)
    print("Epoch ", t, "MSE: ", loss.item())
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

## NLLLoss example

In [47]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)
output.backward()


# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()
# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)
# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
output = loss(m(conv(data)), target)
print(target)
print(m(conv(data)))
print(output.item())
output.backward

tensor([[[2, 1, 2, 0, 1, 3, 2, 0],
         [0, 2, 0, 2, 2, 1, 3, 3],
         [1, 2, 0, 3, 1, 0, 2, 3],
         [0, 2, 2, 3, 1, 1, 3, 1],
         [3, 0, 3, 1, 2, 3, 2, 0],
         [3, 1, 2, 0, 3, 0, 3, 0],
         [2, 1, 0, 1, 3, 0, 3, 2],
         [2, 0, 3, 1, 1, 1, 0, 3]],

        [[3, 2, 2, 2, 2, 3, 3, 1],
         [2, 1, 1, 3, 0, 1, 2, 0],
         [3, 1, 0, 2, 0, 1, 3, 0],
         [3, 2, 3, 0, 1, 3, 0, 3],
         [3, 3, 2, 1, 3, 3, 0, 2],
         [3, 1, 1, 0, 1, 2, 3, 2],
         [2, 1, 3, 1, 2, 1, 0, 1],
         [3, 3, 0, 0, 2, 1, 0, 0]],

        [[2, 0, 0, 1, 0, 0, 0, 0],
         [2, 1, 2, 3, 0, 1, 2, 1],
         [2, 0, 2, 1, 2, 3, 3, 2],
         [0, 1, 2, 0, 0, 2, 0, 2],
         [0, 0, 3, 3, 3, 1, 1, 1],
         [2, 0, 3, 2, 1, 3, 1, 3],
         [1, 1, 1, 0, 2, 0, 1, 0],
         [1, 3, 1, 1, 3, 3, 0, 3]],

        [[3, 1, 2, 2, 3, 2, 1, 2],
         [1, 1, 3, 0, 1, 1, 2, 0],
         [0, 1, 2, 0, 3, 2, 2, 2],
         [2, 3, 3, 0, 0, 3, 0, 0],
         [1, 0

<bound method Tensor.backward of tensor(1.5111, grad_fn=<NllLoss2DBackward>)>

# Plot